In [ ]:
!unzip /content/drive/MyDrive/shapenet-chairs-pcd.zip
!pip3 install open3d
data_folder = "/content/shapenet-chairs-pcd"

# Preprocessing


**Kdtree helper function to reorder the pointcloud**

In [2]:
import numpy as np
import open3d as o3d
import os
import numpy as np

def kdtree(point_list,l,r, depth: int = 0):
    if  l>=r:
        return None

    k = len(point_list[0])  # assumes all points have the same dimension
    
    axis = depth % k
    
    temp = point_list[l:r].copy()
    point_list[l:r] = temp[np.argsort(temp[:, axis])] #sorting the array along axis
    median = l+(r-l) // 2
    
    
    kdtree(point_list,l,median, depth + 1),  #build the left tree
    kdtree(point_list,median + 1,r, depth + 1) #build the right tree

def main():
    """Example usage"""
    array = np.array([(7, 2), (5, 4), (9, 6), (4, 7), (8, 1), (2, 3),(2,7),(6,4),(3,9),(9,8),(12,3)])
    kdtree(array, 0,len(array))
    print(array)
if __name__ == "__main__":
    main()

[[ 2  3]
 [ 5  4]
 [ 2  7]
 [ 3  9]
 [ 4  7]
 [ 6  4]
 [ 7  2]
 [ 8  1]
 [12  3]
 [ 9  6]
 [ 9  8]]


**Reordering Function**

In [3]:
def reordering_pcd(pcd):
  array = np.array(pcd.points)
  kdtree(array,0,len(array)) 
  pcd.points = o3d.utility.Vector3dVector(array)
  return pcd

# new_pcd = reordering_pcd(pcd)
# print(np.asarray(new_pcd.points))




In [4]:
import glob
pcd_files = glob.glob(os.path.join(data_folder,"*.pcd"))
print(len(pcd_files))

5000


**Creating P matrix with 3NxS dimensions**

In [5]:
P = []
N = 1000
for file in pcd_files:
  pcd = reordering_pcd(o3d.io.read_point_cloud(file)) #reordering using kdtree
  pcd_array = np.asarray(pcd.points).reshape(3*N,1) #converting to (3N,1)
  P.append(pcd_array)

P = np.concatenate(P,axis=1) #converting to (3N,S)
print(P.shape)

(3000, 5000)


**PCA**

In [6]:
from sklearn.decomposition import PCA
basis = 100 #100 shape basis
pca = PCA(basis) 
pca.fit(P)
# print(pca.components_.shape) 
# print(pca.explained_variance_.shape)
V = pca.transform(P) #Projections
coeffs = pca.components_.T
print(V.shape)
print(coeffs.shape)

(3000, 100)
(5000, 100)


# Model and Dataloader

**Dataloader**

In [10]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim

class RealCoeff(Dataset):
  def __init__(self, coeffs):
      self.coeffs = coeffs 

  def __len__(self):
      return len(self.coeffs)

  def __getitem__(self, idx):
      return self.coeffs[idx]

batch_size = 16
RealData = RealCoeff(torch.from_numpy(coeffs))
RealDataLoader = torch.utils.data.DataLoader(RealData, batch_size = batch_size,shuffle = True)

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print("Device: ", device)


Device:  cuda:0


**Generator and Discriminator models**

In [11]:
class Generator(nn.Module):
    def __init__(self,input_size = 100,output_size = 100):
      super(Generator, self).__init__()
      self.L1 = nn.Sequential(nn.Linear(input_size, 100),nn.BatchNorm1d(100),nn.ReLU())
      self.L2 = nn.Sequential(nn.Linear(100, 100),nn.BatchNorm1d(100),nn.ReLU())
      self.L3 = nn.Sequential(nn.Linear(100, 100),nn.BatchNorm1d(100),nn.ReLU())
      self.L4 = nn.Sequential(nn.Linear(100, 100),nn.BatchNorm1d(100),nn.ReLU())
      self.L5 = nn.Linear(100,output_size)
    def forward(self, input):
      l1 = self.L1(input)
      l2 = self.L2(l1)
      l3 = self.L3(l2)
      #l4 = self.L4(l3)
      output = self.L5(l3)
      return output

class Discriminator(nn.Module):
    def __init__(self,input_size = 100,output_size = 1):
      super(Discriminator, self).__init__()
      self.L1 = nn.Sequential(nn.Linear(input_size, 100),nn.BatchNorm1d(100),nn.LeakyReLU())
      self.L2 = nn.Sequential(nn.Linear(100, 100),nn.BatchNorm1d(100),nn.LeakyReLU())
      self.L3 = nn.Sequential(nn.Linear(100, 100),nn.BatchNorm1d(100),nn.LeakyReLU())
      self.L4 = nn.Sequential(nn.Linear(100, 100),nn.BatchNorm1d(100),nn.LeakyReLU())
      self.output_layer = nn.Sequential(nn.Linear(100,output_size),nn.Sigmoid())
    def forward(self, input):
      l1 = self.L1(input)
      l2 = self.L2(l1)
      l3 = self.L3(l2)
      l4 = self.L4(l3)
      output = self.output_layer(l4)
      return l1,l2,l3,l4,output



# Training

In [14]:
#Hyperparameters
lr_d = 0.0001
lr_g = 0.0025
beta1 = 0.5
num_epochs = 40

# Initialize BCELoss function
criterion1 = nn.BCELoss()
criterion2 = nn.MSELoss()
# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(batch_size,100, device=device)
print(fixed_noise.shape)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

netD = Discriminator().to(device)
netG = Generator().to(device)
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr_d, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr_g, betas=(beta1, 0.999))

torch.Size([16, 100])


**Normal Training Loop, with conventional Loss function for Generator**

In [15]:
# Training Loop

# Lists to keep track of progress
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(RealDataLoader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data.float().to(device)
        # print(real_cpu.shape)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        _,_,_,_,output = netD(real_cpu)
        output = output.view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion1(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, 100, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        _,_,_,_,output = netD(fake.detach())
        output = output.view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion1(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D, check if the accuracy of the Discriminator is below 80%
        DAcc = (D_x + 1-D_G_z1)/2
        if DAcc<0.8:
          optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        _,_,_,_,output = netD(fake)
        output = output.view(-1)
        # Calculate G's loss based on this output
        errG = criterion1(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(RealDataLoader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        # if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
        #     with torch.no_grad():
        #         fake = netG(fixed_noise).detach().cpu()
        #     img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Starting Training Loop...
[0/40][0/313]	Loss_D: 1.3931	Loss_G: 0.6957	D(x): 0.5146	D(G(z)): 0.5094 / 0.5039
[0/40][100/313]	Loss_D: 1.4062	Loss_G: 0.6909	D(x): 0.4983	D(G(z)): 0.5052 / 0.5027
[0/40][200/313]	Loss_D: 1.3772	Loss_G: 0.7081	D(x): 0.5054	D(G(z)): 0.4968 / 0.4947
[0/40][300/313]	Loss_D: 1.4023	Loss_G: 0.6892	D(x): 0.5005	D(G(z)): 0.5049 / 0.5024
[1/40][0/313]	Loss_D: 1.4037	Loss_G: 0.6907	D(x): 0.4979	D(G(z)): 0.5040 / 0.5022
[1/40][100/313]	Loss_D: 1.3970	Loss_G: 0.6978	D(x): 0.5006	D(G(z)): 0.5019 / 0.4994
[1/40][200/313]	Loss_D: 1.3826	Loss_G: 0.7072	D(x): 0.5016	D(G(z)): 0.4964 / 0.4939
[1/40][300/313]	Loss_D: 1.4022	Loss_G: 0.6964	D(x): 0.4967	D(G(z)): 0.5018 / 0.4998
[2/40][0/313]	Loss_D: 1.3963	Loss_G: 0.6940	D(x): 0.4981	D(G(z)): 0.5021 / 0.5000
[2/40][100/313]	Loss_D: 1.3744	Loss_G: 0.7048	D(x): 0.5049	D(G(z)): 0.4973 / 0.4951
[2/40][200/313]	Loss_D: 1.3810	Loss_G: 0.6949	D(x): 0.5061	D(G(z)): 0.5015 / 0.4998
[2/40][300/313]	Loss_D: 1.3803	Loss_G: 0.7029	D(x): 0.50

In [16]:
# function to get the covariance tensor
def cov(tensor, rowvar=False, bias=False):
    """Estimate a covariance matrix (np.cov)"""
    tensor = tensor if rowvar else tensor.transpose(-1, -2)
    tensor = tensor - tensor.mean(dim=-1, keepdim=True)
    factor = 1 / (tensor.shape[-1] - int(not bool(bias)))
    return factor * tensor @ tensor.transpose(-1, -2).conj()

In [17]:
netD = Discriminator().to(device)
netG = Generator().to(device)
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr_d, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr_g, betas=(beta1, 0.999))

**Modified Training with new loss function for Generator**

In [19]:
# Training Loop

# Lists to keep track of progress
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(RealDataLoader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real = data.float().to(device)
        # print(real.shape)
        b_size = real.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        _,_,_,_,output = netD(real)
        output = output.view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion1(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, 100, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        _,_,_,_,output = netD(fake.detach())
        output = output.view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion1(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        DAcc = (D_x + 1-D_G_z1)/2
        if DAcc<0.8:
          optimizerD.step()

        ############################
        # (2) Update G network: 
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        l1_fake,l2_fake,l3_fake,l4_fake,output = netD(fake)
        l1_real,l2_real,l3_real,l4_real,_ = netD(real)
        output = output.view(-1)

        # Calculate G's loss based on this output
        # G's loss = errG_mean + errG_cov
        # errG_mean = rmse(mean(activations(real)),mean(activations(fake)))
        # errG_cov = rmse(mean(activations(real)),mean(activations(fake)))
        # choose 3rd layer as an intermidiate layer as the activation to compute the loss
        errG_mean = criterion2(l3_fake.mean(axis=1), l3_real.mean(axis=1))
        errG_cov = criterion2(cov(l3_fake),cov(l3_real))
        # Calculate gradients for G
        errG = errG_mean + errG_cov
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(RealDataLoader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        iters += 1

Starting Training Loop...
[0/40][0/313]	Loss_D: 1.3596	Loss_G: 0.0245	D(x): 0.5066	D(G(z)): 0.4901 / 0.4870
[0/40][100/313]	Loss_D: 1.3551	Loss_G: 0.0335	D(x): 0.5039	D(G(z)): 0.4846 / 0.4819
[0/40][200/313]	Loss_D: 1.3802	Loss_G: 0.0299	D(x): 0.4978	D(G(z)): 0.4915 / 0.4877
[0/40][300/313]	Loss_D: 1.3550	Loss_G: 0.0276	D(x): 0.5037	D(G(z)): 0.4849 / 0.4821
[1/40][0/313]	Loss_D: 1.3492	Loss_G: 0.0338	D(x): 0.5077	D(G(z)): 0.4868 / 0.4837
[1/40][100/313]	Loss_D: 1.3577	Loss_G: 0.0273	D(x): 0.5083	D(G(z)): 0.4905 / 0.4878
[1/40][200/313]	Loss_D: 1.3381	Loss_G: 0.0342	D(x): 0.5115	D(G(z)): 0.4834 / 0.4800
[1/40][300/313]	Loss_D: 1.3329	Loss_G: 0.0244	D(x): 0.5123	D(G(z)): 0.4810 / 0.4776
[2/40][0/313]	Loss_D: 1.3579	Loss_G: 0.0203	D(x): 0.5063	D(G(z)): 0.4877 / 0.4848
[2/40][100/313]	Loss_D: 1.3308	Loss_G: 0.0370	D(x): 0.5144	D(G(z)): 0.4827 / 0.4791
[2/40][200/313]	Loss_D: 1.3510	Loss_G: 0.0426	D(x): 0.5095	D(G(z)): 0.4857 / 0.4818
[2/40][300/313]	Loss_D: 1.2921	Loss_G: 0.0279	D(x): 0.52

# Visualization


**Visualize Training Data**

In [ ]:

import plotly.graph_objects as go
import plotly.express as px
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig


def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

In [ ]:
pcd = reordering_pcd(o3d.io.read_point_cloud(pcd_files[0]))
pcd_array = np.array(pcd.points)
kdtree(pcd_array,0,1000)
x = pcd_array[:,0]
y = pcd_array[:,1]
z = pcd_array[:,2]
pcshow(x,y,z)

**Visualize Result**

In [ ]:
noise = torch.randn(64, 100, device=device)
fake = netG(noise).detach().cpu()
print(fake.shape)

torch.Size([64, 100])


In [ ]:
points = []
proj = torch.from_numpy(V).float()
for i in range(len(fake)):
  temp = fake[i]
  temp = temp.reshape(-1,1)
  out = torch.matmul(proj,temp).numpy()
  points.append(out)

In [ ]:
temp = points[20]
temp = temp.reshape(1000,3)

In [ ]:
pcshow(temp[:,0],temp[:,1],temp[:,2])

# Iterative Point Ordering


**Optimizing Point Ordering**

In [ ]:
pcd_arrays = list()
for file in pcd_files:
  pcd_array = np.array(reordering_pcd(o3d.io.read_point_cloud(file)).points)
  pcd_arrays.append(pcd_array)


In [ ]:
import random
from tqdm.notebook import tqdm

# select two points at random, swap, check if reprojection error reduces, return swapped array
def shuffle_order(array,mean,u):
  old_array = array.copy()
  l=0; r=0
  while(l==r):
    l = random.randint(0,999)
    r = random.randint(0,999)
  temp = array[l,:].copy()
  array[l,:] = array[r,:].copy()
  array[r,:] = temp.copy()
  array = array.reshape(3000,1)
  old_array = old_array.reshape(3000,1)
  u = u.reshape(-1,1)
  #reproj = np.dot(np.dot(array-mean,u.T),u) + mean
  reproj = np.dot(array-mean,np.dot(u.T,u)) + mean
  #reproj_old = np.dot(np.dot(old_array-mean,u.T),u) + mean
  reproj_old = np.dot(old_array-mean,np.dot(u.T,u)) + mean
  error = np.mean((array - reproj)**2)
  error_old = np.mean((old_array - reproj_old)**2)
  if error<error_old:
    # print("yes")
    return array.reshape(1000,3),error
  else:
    return old_array.reshape(1000,3),error_old

# Calculate PCA 
def calculate_PCA(array_list):
  new_list = []
  for i in range(len(array_list)):
    new_list.append(array_list[i].reshape(3000,1))
  P = np.concatenate(new_list,axis=1)
  basis = 100 #100 shape basis
  pca = PCA(basis) 
  pca.fit(P)
  V = pca.transform(P) #Projections
  coeffs = pca.components_.T
  return coeffs, V

# Mean across all the arrays, result size is 3NX1
def calculate_mean(array_list):
  sumation = np.zeros(array_list[0].shape)
  for array in array_list:
    sumation+=array
  sumation/= len(array_list)
  return sumation.reshape(3000,1)

# Function that calculates reprojection error for one shape
def calculate_error(array,u,mean):
  u = u.reshape(-1,1)
  array = array.reshape(3000,1)
  reproj = np.dot(array-mean,np.dot(u.T,u)) + mean
  error = np.mean((array - reproj)**2)
  return error


iterations = 100
K = 10000
mean = calculate_mean(pcd_arrays)
coeffs,V = calculate_PCA(pcd_arrays)
for i in range(iterations):
  # print("iteration number: ",i)
  total_error = 0
  for j in tqdm(range(len(pcd_arrays))):
    for k in range(K):
      pcd_arrays[j], error = shuffle_order(pcd_arrays[j],mean,coeffs[j])
  for l in range(len(pcd_arrays)):
    total_error+=calculate_error(pcd_arrays[l],coeffs[l],mean)
  print("error in iteration %d is %f"%(i,total_error))
  coeffs, V = calculate_PCA(pcd_arrays)
  mean = calculate_mean(pcd_arrays)

# print(coeffs)
# print(V)


error in iteration 0 is 43.445359



error in iteration 1 is 42.820477



error in iteration 2 is 42.306123



error in iteration 3 is 41.877463



error in iteration 4 is 41.508089



error in iteration 5 is 41.180174
